In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 k

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-26 00:06:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.50MB/s    in 0.2s    

2022-05-26 00:06:32 (4.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in the Amazon Electronics review from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option("header", "true").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep="\t", timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [5]:
# Drop duplicates and incomplete rows
print(f"Row count from file: {df.count()}")
df = df.dropna()
print(f"Row count after Nan are removed: {df.count()}")
df = df.dropDuplicates()
print(f"Row count after Nan and duplicates are removed: {df.count()}")

Row count from file: 960872
Row count after Nan are removed: 960679
Row count after Nan and duplicates are removed: 960679


In [6]:
# Examine the schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Import struct fields that will be used
from pyspark.sql.types import (
    StringType, BooleanType, IntegerType, FloatType, DateType
)

In [8]:
# Change the data type of the "star_rating" and "review_date" columns
df2 = df.select(df.marketplace,
                df.customer_id,
                df.review_id,
                df.product_id,
                df.product_title,
                df.product_parent,
                df.product_category,
                (df.star_rating.cast(IntegerType())).alias("star_rating"),
                df.helpful_votes,
                df.total_votes,
                df.vine,
                df.verified_purchase,
                df.review_headline,
                df.review_body,
                (df.review_date.cast(DateType())).alias("review_date"),
)

In [9]:
# Examine the schema
df2.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# Create review_id_table dataframe
review_id_table_df = df2.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
# Drop duplicates
review_id_table_df = review_id_table_df.dropDuplicates()
# Preview dataframe
review_id_table_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O51G2D52W9DD|     112967|B00G9X76DG|      86861596| 2015-08-31|
|R2M6EF6NI4VV6I|    3669388|B00A6OEWA4|     985088364| 2015-08-31|
| RE6WK260HJKO0|   50230393|B00LP7HSAW|     824548355| 2015-08-31|
|R3MAHOLAEXF0BH|   38604950|B002ORU02K|     473768853| 2015-08-31|
| RBP8V63QHJD0Z|     260271|B00MA9D9NY|     208327980| 2015-08-31|
|R1LOLERHCZVOC4|   14600611|B00IAGERCI|     903001693| 2015-08-30|
|R32R7ANZZTRY48|   24036078|B00IO757BI|     731694725| 2015-08-30|
|R39H359U771N4C|   25998108|B00ODEC8G8|     692782008| 2015-08-30|
|R371WMDOMFJVRJ|    6635197|B00JSYCT7Q|     220039697| 2015-08-29|
|R3HUA0D9K4SWGZ|   13153840|B00ECVW2KE|     135630862| 2015-08-29|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [11]:
# Examine the schema
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
# Create products dataframe
products_df = df2.select(["product_id", "product_title"])
# Drop duplicates
products_df = products_df.dropDuplicates()
# Preview dataframe
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EVX7V1I|Game Time Women's...|
|B009S4DODY|XOXO Women's XO55...|
|B00LBKXQRW|Anne Klein Women'...|
|B0009P679Y|Invicta Men's 993...|
|B00DHF30RU|M&c Women's | Cla...|
|B00NIDA43Y|GuTe Classic Skel...|
|B008EQDDPQ|Nautica Men's N13...|
|B004VRBZ66|Timex Men's T2N63...|
|B009BEO81I|        Fossil Riley|
|B008B39MTI|XOXO Women's XO55...|
+----------+--------------------+
only showing top 10 rows



In [13]:
# Examine the schema
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
# Create vine_table dataframe
vine_table_df = df2.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
# Preview dataframe
vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OPQYUMAHS|          1|            4|          5|   N|
|R1010XUX2AIF10|          5|            0|          0|   N|
|R102DONPK6YEGN|          5|            0|          0|   N|
|R102N48HXFM9WY|          5|            0|          0|   N|
|R1060WX8UX26JV|          2|            0|          0|   N|
|R106D023EFXYHQ|          5|            0|          0|   N|
|R107WJY4V4IZ35|          5|            0|          0|   N|
|R10B275CIJ1QSG|          4|            0|          0|   N|
|R10BIT0LX7BBYD|          4|            1|          2|   N|
|R10C1EZ2LJLSKN|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [15]:
# Examine the schema
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [16]:
# Create customers dataframe
temp = df2.select(["customer_id"])
customers_df = temp.groupBy("customer_id").count().alias("customer_count")
# Rename "count" column to "customer_count"
customers_df = customers_df.withColumnRenamed("count","customer_count")
# Change the data type of the "customer_count" column
customers_df2 = customers_df.select(customers_df.customer_id,
                                    (customers_df.customer_count.cast(IntegerType())).alias("customer_count"),
                                    )
# Drop duplicates
customers_df2 = customers_df2.dropDuplicates()
# Preview dataframe
customers_df2.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12937860|             1|
|   14327548|             4|
|   22158149|             5|
|   12828562|             2|
|    4089580|             1|
|   20097895|             1|
|    2422277|             4|
|   35523976|             2|
|   14051742|             1|
|   14404180|             5|
+-----------+--------------+
only showing top 10 rows



In [17]:
# Examine the schema
customers_df2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [18]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/amazon_reviews_watches"
config = {"user":"<insert username>",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [19]:
# Write DataFrame to RDS
review_id_table_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [20]:
# Write DataFrame to RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [21]:
# Write DataFrame to RDS
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)

In [22]:
# Write DataFrame to RDS
customers_df2.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)